In [1]:
import sys
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense , Dropout , Activation , Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [2]:
df=pd.read_csv('fer2013.csv')

In [3]:
x_train ,y_train,x_test,y_test = [],[],[],[]

In [4]:
for index,row in df.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            x_train.append(np.array(val,'float32'))
            y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            x_test.append(np.array(val,'float32'))
            y_test.append(row['emotion'])
    except:
        print(f'error occured at index :{index} and row:{row}')


In [5]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width , height = 48, 48

In [6]:
x_train = np.array(x_train,'float32')
y_train = np.array(y_train,'float32')

x_test = np.array(x_test,'float32')
y_test = np.array(y_test,'float32')


In [7]:
y_train = np_utils.to_categorical(y_train , num_classes=num_labels)
y_test = np_utils.to_categorical(y_test,num_classes=num_labels)

In [8]:
x_train -= np.mean(x_train,axis=0)
x_train /= np.std(x_train, axis=0)

x_test -= np.mean(x_test,axis=0)
x_test /= np.std(x_test, axis=0)

In [9]:
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)

In [10]:
model = Sequential()

In [11]:
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(x_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

In [12]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

model.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          shuffle=True)

fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

Epoch 1/30
449/449 [==============================] - 287s 640ms/step - loss: 1.7129 - accuracy: 0.3020 - val_loss: 1.5572 - val_accuracy: 0.3834
Epoch 2/30
449/449 [==============================] - 293s 654ms/step - loss: 1.5172 - accuracy: 0.4073 - val_loss: 1.3924 - val_accuracy: 0.4650
Epoch 3/30
449/449 [==============================] - 298s 665ms/step - loss: 1.4192 - accuracy: 0.4491 - val_loss: 1.3214 - val_accuracy: 0.4862
Epoch 4/30
449/449 [==============================] - 292s 650ms/step - loss: 1.3478 - accuracy: 0.4817 - val_loss: 1.3014 - val_accuracy: 0.5001
Epoch 5/30
449/449 [==============================] - 302s 674ms/step - loss: 1.3130 - accuracy: 0.4933 - val_loss: 1.2849 - val_accuracy: 0.5077
Epoch 6/30
449/449 [==============================] - 319s 711ms/step - loss: 1.2706 - accuracy: 0.5122 - val_loss: 1.2356 - val_accuracy: 0.5233
Epoch 7/30
449/449 [==============================] - 308s 686ms/step - loss: 1.2352 - accuracy: 0.5271 - val_loss: 1.2118 -